In [1]:
from match import get_track_info, get_data, get_artist_info, find_shared, get_image, get_genres, recommend_artists, recommend_tracks, get_value
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [2]:
pidx = '2b08l28qJ61zKFztCRrNOl'
pid1 = '5t1y9F77hIHGFIUJbdy2PH'
pid2 = '2b08l28qJ61zKFztCRrNOl'
uid = 'kli-17'

In [3]:
def g_audio_features(track_ids):
    r = []
    for track in track_ids:
        data = get_data(track, 'audio-features')
        r.append(data)
    return r

In [4]:
FEATURES = ['track_titles', 'artist_ids', 'artist_name', 'danceability', 'energy', 'acousticness', 'mode', 'valence', 'loudness', 'tempo', 'liveness', 'key', 'instrumentalness', 'likes']

In [5]:
def create_df(pid, like_flag):
    pl = get_data(pid, 'playlists')
    track_ids = get_track_info(pl, 'id')
    artist_ids = get_artist_info(pl, 'id')
    artist_names = get_artist_info(pl, 'name')
    track_titles = get_track_info(pl, 'name')

    features = g_audio_features(track_ids)
    df = pd.DataFrame(data=features, columns=features[0].keys())

    likes = []
    for i in range(0, len(df.index)):
        likes.append(0) if like_flag == 0 else likes.append(1)
    
    df['track_titles'] = track_titles
    df['artist_name'] = artist_names
    df['artist_ids'] = artist_ids
    df['likes'] = likes

    return df[FEATURES]

In [6]:
df1y = create_df(pid1, 1)
df1x = create_df(pidx, 0)
df2 = create_df(pid2, 1)
df1 = pd.concat([df1y, df1x])

In [7]:
REDUCED_FEATURES = ['danceability', 'energy', 'acousticness', 'valence', 'loudness', 'tempo', 'liveness', 'instrumentalness', 'key', 'mode']

In [8]:
def find_compatibility(X_train, y_train, X_test, y_test):
    dtc = DecisionTreeClassifier()
    dtc.fit(X_train, y_train)
    return dtc.score(X_test, y_test)

In [9]:
c = find_compatibility(df1[REDUCED_FEATURES], df1['likes'], df2[REDUCED_FEATURES], df2['likes']) * 100